# Lake model continued

In the previous week you used the lake problem as a means of getting aquinted with the workbench. In this assignment we will continue with the lake problem, focussing explicitly on using it for open exploration. You can use the second part of [this tutoria](https://emaworkbench.readthedocs.io/en/latest/indepth_tutorial/open-exploration.html) for help.

**It is paramount that you are using the lake problem with 100 decision variables, rather than the one found on the website with the seperate anthropogenic release decision**

## Apply sensitivity analysis
There is substantial support in the ema_workbench for global sensitivity. For this, the workbench relies on [SALib](https://salib.readthedocs.io/en/latest/) and feature scoring which is a machine learning alternative for global sensitivity analysis. 


1. Apply Sobol with 3 seperate release policies (0, 0.05, and 0.1) and analyse the results for each release policy seperately focusing on the reliability objective. Do the sensitivities change depending on the release policy? Can you explain why or why not?

*hint: you can use sobol sampling for the uncertainties, and set policies to a list with the 3 different release policies. Next, for the analysis using logical indexing on the experiment.policy column you can select the results for each seperate release policy and apply sobol to each of the three seperate release policies. If this sounds too complicated, just do it on each release policy seperately.*



In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from lakemodel_function import lake_problem as lake_model

from scipy.integrate import odeint
from ema_workbench import Model, RealParameter, TimeSeriesOutcome, ScalarOutcome, Policy

model = Model('LAKEMODEL', function=lake_model)

model.uncertainties = [RealParameter('b', 0.1, 0.45),
                       RealParameter('mean', 0.01, 0.05),
                       RealParameter('q', 2, 4.5),
                       RealParameter("stdev", 0.001, 0.005),
                       RealParameter('delta', 0.93, 0.99)]

model.levers = [RealParameter(f'l{i}', 0,0.1) for i in range(100)]


model.outcomes = [ScalarOutcome('max_P'),
                  ScalarOutcome('utility'),
                  ScalarOutcome('inertia'),
                  ScalarOutcome('reliability')]

In [4]:
from SALib.analyze import sobol
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from ema_workbench import (MultiprocessingEvaluator, SequentialEvaluator ,ema_logging,
                           perform_experiments)
ema_logging.log_to_stderr(ema_logging.INFO)

with SequentialEvaluator(model) as evaluator:
    sa_results = evaluator.perform_experiments(scenarios = 50 ,
                                               uncertainty_sampling='sobol', policies=3)


[MainProcess/INFO] performing 600 scenarios * 3 policies * 1 model(s) = 1800 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 180 cases completed
[MainProcess/INFO] 360 cases completed
[MainProcess/INFO] 540 cases completed
[MainProcess/INFO] 720 cases completed
[MainProcess/INFO] 900 cases completed
[MainProcess/INFO] 1080 cases completed
[MainProcess/INFO] 1260 cases completed
[MainProcess/INFO] 1440 cases completed
[MainProcess/INFO] 1620 cases completed
[MainProcess/INFO] 1800 cases completed
[MainProcess/INFO] experiments finished


In [11]:
experiments, outcomes = sa_results

problem_1 = get_SALib_problem(model.uncertainties)
Si_1 = sobol.analyze(problem_1, outcomes['max_P'],
                   calc_second_order=True, print_to_console=False)

In [14]:
Si_filter_1 = {k:Si_1[k] for k in ['ST','ST_conf','S1','S1_conf']}
Si_df_1 = pd.DataFrame(Si_filter_1, index = problem_1['names'])
Si_df_1

NameError: name 'problem' is not defined

2. Repeat the above analysis for the 3 release policies but now with extra trees feature scoring and for all outcomes of interest. As a bonus, use the sobol experiment results as input for extra trees, and compare the results with those resulting from latin hypercube sampling.

*hint: you can use [seaborn heatmaps](https://seaborn.pydata.org/generated/seaborn.heatmap.html) for a nice figure of the results*


b     delta      mean         q     stdev        l0        l1  \
0     0.176904  0.935801  0.030742  3.691895  0.002121  0.051048  0.008422   
1     0.417529  0.935801  0.030742  3.691895  0.002121  0.051048  0.008422   
2     0.176904  0.932754  0.030742  3.691895  0.002121  0.051048  0.008422   
3     0.176904  0.935801  0.045977  3.691895  0.002121  0.051048  0.008422   
4     0.176904  0.935801  0.030742  3.252441  0.002121  0.051048  0.008422   
...        ...       ...       ...       ...       ...       ...       ...   
1795  0.428296  0.976963  0.019746  3.820068  0.002311  0.079333  0.073568   
1796  0.428296  0.946377  0.029160  3.820068  0.002311  0.079333  0.073568   
1797  0.428296  0.946377  0.019746  2.357666  0.002311  0.079333  0.073568   
1798  0.428296  0.946377  0.019746  3.820068  0.004354  0.079333  0.073568   
1799  0.428296  0.946377  0.019746  3.820068  0.002311  0.079333  0.073568   

           l10       l11       l12  ...       l93       l94       l95  \
0  